In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('BitcoinHeistData.csv')
df = df.dropna()  ## dropping null values

df  = df.sample(frac=1)

In [3]:
## Splitting into train test and validation datatests
train_size = 0.7
valid_size=0.15
train_index = int(len(df)*train_size)
df_train = df[0:train_index]
df_rem = df[train_index:]
valid_index = int(len(df)*valid_size)

df_valid = df[train_index:train_index+valid_index]
df_test = df[train_index+valid_index:]
df


,address,year,day,length,weight,count,looped,neighbors,income,label
1875414,19JDMrubNSMKWMd4aefexqt5SSiGzYnhtq,2016,19,12,2.187500,4,0,2,135200000.0,white
1192664,1JvusjLkvuBYCfgxw6KwX8HnDVb3pjSjZJ,2014,66,144,0.013911,1550,1208,2,45128303.0,white
1080058,1FdCjMvaPs8njsLKgjZekQDhBKLGX1Q37Z,2013,319,4,0.250000,2,0,2,101351351.0,white
291877,1FZkDZdhReaCVnTZ2eDZTw7qhTqH9jkqaU,2011,261,6,0.145483,27,26,3,100908523.0,white
1442389,1Di1T4H7FR67Pq2YubkESKkEA5dNE9DuhH,2014,316,144,0.088939,3642,0,2,297208000.0,white
...,...,...,...,...,...,...,...,...,...,...
2566014,1N3sbxkwwzfg65u32Tpz5x3DuVNZ1FWGKR,2017,345,16,0.015625,1,0,2,59080704.0,white
2756183,1EHocpqQ8wwk623mhwVFG5om6njqyLb2Pe,2018,170,0,0.500000,1,0,1,64740171.0,white
1920539,1DjcZ9GdvLZuhAhctbQggfohYTWf21XN8H,2016,64,144,0.000905,5446,5378,2,31100000.0,white
1852642,1NAG5f4iuUhgqA8Qhm9coRundm3Yvh3NcF,2015,361,0,1.000000,1,0,1,151178747.0,white


In [4]:
X_train, y_train = df_train.drop(columns=['label' , 'address']).copy(), df_train['label'].copy()
X_valid, y_valid = df_valid.drop(columns=['label' , 'address']).copy(), df_valid['label'].copy()
X_test, y_test = df_test.drop(columns=['label' , 'address']).copy(), df_test['label'].copy()

## Spitting into X and Y data

#### a) Train a decision tree using both the Gini index and the Entropy

##### Max depth 4

In [5]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=4 , criterion='entropy') ## criteria has by default gini
clf = clf.fit(X_train , y_train)
predictions = clf.predict(X_test)
clf.score(X_test , y_test)

## Reference taken from scikit-learn.org


0.985945335606826

##### Max depth 8

In [6]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=8 , criterion='entropy') ## criteria has by default gini
clf = clf.fit(X_train , y_train)
predictions = clf.predict(X_test)
clf.score(X_test , y_test)

## Reference taken from scikit-learn.org


0.9862447600718619

##### Max depth 10

In [7]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=10 , criterion='entropy') ## criteria has by default gini
clf = clf.fit(X_train , y_train)
predictions = clf.predict(X_test)
clf.score(X_test , y_test)

## Reference taken from scikit-learn.org


0.9874447436149447

##### Max depth 15

In [8]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=15 , criterion='entropy') ## criteria has by default gini
clf = clf.fit(X_train , y_train)
predictions = clf.predict(X_test)
clf.score(X_test , y_test)

## Reference taken from scikit-learn.org


0.9882333042289706

##### Max depth 20

In [9]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=20 , criterion='entropy') ## criteria has by default gini
clf = clf.fit(X_train , y_train)
predictions = clf.predict(X_test)
clf.score(X_test , y_test)

## Reference taken from scikit-learn.org


0.9863339017064909

We are getting the following results 

Max depth = 4   Accuracy = 0.9859590497044612

Max depth = 8   Accuracy = 0.9862447600718619

Max depth = 10  Accuracy = 0.9874424579320055

Max depth = 15  Accuracy = 0.9883247315465388

Max depth = 20  Accuracy = 0.9862950450965244


We are getting best result with  Max Depth = 15 

#### b) Ensembling is a method to combine multiple not-so-good models to get a better performing model.

In [10]:
import numpy as np

## We have to create 100 decision stumps
stumps=[]
for i in range(100):
    stumps.append(DecisionTreeClassifier(criterion="entropy",max_depth=3))

## Now we have to train it on randomly selected 50% of data


for i in range(100):
    X_train_frac=X_train.sample(frac=0.5)
    y_train_frac=y_train.loc[X_train_frac.index]
    stumps[i].fit(X_train,y_train)
## Now we have Trained 
## Now we have to check the prediction




In [11]:
import numpy as np

predicted_samples=[]
## The list of predicted samples
for i in range(100):
    predicts=stumps[i].predict(X_valid)
    predicted_samples.append(predicts)

new_predicted_samples=[]   ## List of new predicted Sample ( after majority voting of output of stumps)

##  Majority voting of output of stumps
for i in range(len(predicted_samples[0])):
    max_val={}   # creating a dictionary to store the key value pair of y 
    for j in predicted_samples:
        if(j[i] in max_val):
            max_val[j[i]]+=1
        else:
            max_val[j[i]]=1
    max_key=max(max_val, key= lambda x: max_val[x])
    new_predicted_samples.append(max_key)   ## appeding the y with maximum value of dictionary

accuracy=np.sum(np.array(new_predicted_samples)==np.array(y_valid.to_list()))/len(y_valid)   ## Calculating Accuracy 

print("Accuracy obtained is "+str(accuracy))

Accuracy obtained is 0.9858904147162083


The accuracy coming from Ensembling by creating 100 different decision stumps on 50% data and predicting the test samples’ labels by taking a majority vote of the output of the stumps is 0.9857715586600351

#### C) Use the Adaboost algorithm on the above dataset and report the testing accuracy.

In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=15)  , n_estimators=4 )
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## Reference taken from scikit-learn.org


0.9668713114791568

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=15) , n_estimators=8 )
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## Reference taken from scikit-learn.org


0.9737169318820771

In [14]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=15) ,    n_estimators=10)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
## Reference taken from scikit-learn.org


0.9765900353366582

In [17]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=15) , n_estimators=15)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## Reference taken from scikit-learn.org


0.9801739861853324

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier( DecisionTreeClassifier(max_depth=15)  , n_estimators=20)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## Reference taken from scikit-learn.org


0.984343071866443

#### Using AdaBoost
##### Using Max Depth = 15

n estimators = 4  Accuracy = 0.9723615218991283

n estimators = 8  Accuracy = 0.9765351789461173

n estimators = 10  Accuracy = 0.9801031300142169

n estimators = 15 Accuracy = 0.8836747381750193

n estimators = 20  Accuracy = 0.9844184994034367

We are getting the best result with n estimators = 20 and when max depth = 15

#### Comparison between Random Forest and Adaboost

The best Accuracy that we got with Random Forest was 0.9883247315465388

With Adaboost we are getting Accuracy 0.9844184994034367